<a href="https://colab.research.google.com/github/lphansen/RiskUncertaintyValue/blob/main/quickguide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this notebook 

- Section 1 provides a five-minute guide to solving the DSGE model using the expansion suite, as well as how to 
    - compute shock elasticities and IRF
    - approximate and simulate variables based on model solutions. 


- Section 2 provides examples of `LinQuadVar` computations, including
    - constructions, operations, and evaluations
    - adjusting time periods
    - computing expectations


In [4]:
import os
import sys
workdir = os.getcwd()
# !git clone https://github.com/lphansen/RiskUncertaintyValue # Uncomment this when running on Google Colab
# workdir = os.getcwd() + '/RiskUncertaintyValue'             # Uncomment this when running on Google Colab
sys.path.insert(0, workdir+'/src')                        
import numpy as np
import autograd.numpy as anp
from scipy import optimize
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=200)
from IPython.display import display, HTML
from BY_example_sol import disp
display(HTML("<style>.container { width:97% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")

from lin_quad import LinQuadVar
from lin_quad_util import E, concat, next_period, cal_E_ww, lq_sum, N_tilde_measure, E_N_tp1
from uncertain_expansion import uncertain_expansion
np.set_printoptions(suppress=True)

# 1. Five-minute guide to solving the DSGE model using the Expansion Suite

Suppose we would like to solve an adjustment cost model. Its equilibrium conditions include
- Three forward-looking conditions
$$
\begin{aligned}
\frac{C_t}{K_t} + \frac{I_t}{K_t}  &= {\mathbf a}\\
\mathbb{E}\left[\left(\frac{S_{t+1}}{S_t}\right) \frac {MK_{t+1}}{MC_{t+1}} \frac{K_{t+1}}{K_t} \frac{1}{1+\theta_2\left(\frac{I_t}{K_t}\right)} \mid \mathfrak{A}_t\right] - 1  &= 0\\
\mathbb{E}\left[\left(\frac{S_{t+1}}{S_t}\right) \frac {MK_{t+1}}{MC_{t+1}} \frac{K_{t+1}}{K_t} \frac{1
+(\theta_2-1)\frac{I_t}{K_t}}{1+\theta_2\left(\frac{I_t}{K_t}\right)} \mid \mathfrak{A}_t\right] - \frac{MK_t}{MC_t}+{\mathbf a}&= 0
\end{aligned}
$$

- Two state-evolution equations
$$
\begin{aligned}
\frac{K_{t+1}}{K_t} &=  \left[1 + \theta_2 \left({\frac {I_t} {K_t}}\right) \right]^{\theta_1} \exp( - \alpha_k + Z_t - {\frac 1 2} \mid \sigma_k \mid^2  + \sigma_k\cdot W_{t+1})\\
Z_{t+1} &= {\mathbb A} Z_t + \mathbb{B} W_{t+1}
\end{aligned}
$$

where we have
$$
\begin{aligned}
\frac{S_{t+1}}{S_t}& = \beta \left(\frac{V_{t+1}}{R_t}\right)^{1-\gamma} \left(\frac{V_{t+1}}{R_t}\right)^{\rho-1} \left(\frac{C_{t+1}}{C_t}\right)^{-\rho}
\end{aligned}
$$

The model has one jump variable (In the expansion suite, $J_t$ is used to represent jump variables):
- $\log C_t - \log K_t$ : Log consumption over capital
- $\log I_t - \log K_t$ : Log investment over capital
- $\log MK_t - \log MC_t$ : Log co-state variable

Two state variables (In the expansion suite, $X_t$ is used to represent state variables):
- $\log K_{t} - \log K_{t-1}$ : Log capital growth process
- $Z_{t}$ : Exogenous state

The Expansion Suite only takes **five** necessary inputs to solve the DSGE model.

## 1.1 Equilibrium Condition Function

The Equilibrium Condition Function takes the form of the next cell. 

Note that:

1. At the beginning of this function, we need to specify the parameters and variables used in the equilibrium conditions. 

    - The first components of `Var_t` and `Var_tp1` are fixed as `q_t` or `q_tp1`. (See Exploring Recursive Utility Appendix for details.)
    
    - State variables need to follow jump variables.

    - The number of jump variables (except `q_t` and `q_tp1`) should equal the number of forward-looking equilibrium conditions. 

    - The number of state variables should equal to the number of state evolution equations.

2. Then we specify each equilibrium condition using above specified parameter and variables. 

    - The algorithm requires us to rewrite the equilibrium conditions in the form as equation (1). We need to find what `psi1` is in our equilibrium conditions, specify what  `psi1` look like, and then return it using `psi1` mode. 
    
    - The log change of measure is skipped in the specification of equilibrium conditions.

3. By default (without `mode` specification), the function needs to return the LHS of each equilibrium condition before taking expectation. 

    - The state-evolution equations should follow the forward-looking conditions.

$$
\begin{aligned}
\mathbb{E}[{N_{t+1}^*Q_{t+1}^* \psi_1(X_t, J_t, X_{t+1}, J_{t+1})}|{\mathfrak{A}_t}] - \psi_2(X_t, J_t)  &= 0 \\
  \phi\left(X_t, X_{t+1}, J_t, {\sf q}W_{t+1}, {\sf q} \right) &= 0 
\end{aligned}\tag{1}
$$

- In this simple example, we can rewrite the FOC and co-state equation as 
$$
\begin{aligned}
\mathbb{E}\left[ N_{t+1}^*Q_{t+1}^* \beta\left(\frac{C_{t+1}}{C_t}\right)^{-\rho}\frac {MK_{t+1}}{MC_{t+1}} \frac{K_{t+1}}{K_t} \frac{1}{1+\theta_2\left(\frac{I_t}{K_t}\right)} \mid \mathfrak{A}_t\right] - 1  &= 0\\
\mathbb{E}\left[N_{t+1}^*Q_{t+1}^* \beta\left(\frac{C_{t+1}}{C_t}\right)^{-\rho} \frac {MK_{t+1}}{MC_{t+1}} \frac{K_{t+1}}{K_t} \frac{1
+(\theta_2-1)\frac{I_t}{K_t}}{1+\theta_2\left(\frac{I_t}{K_t}\right)} \mid \mathfrak{A}_t\right] - \frac{MK_t}{MC_t}+{\mathbf a}&= 0\\
\end{aligned}
$$

- where `psi1` will be a three dimensional array 
$$
\psi_1 = 
\begin{bmatrix}
0\\ 
\beta\left(\frac{C_{t+1}}{C_t}\right)^{-\rho}\frac {MK_{t+1}}{MC_{t+1}} \frac{K_{t+1}}{K_t} \frac{1}{1+\theta_2\left(\frac{I_t}{K_t}\right)}\\
\beta\left(\frac{C_{t+1}}{C_t}\right)^{-\rho} \frac {MK_{t+1}}{MC_{t+1}} \frac{K_{t+1}}{K_t} \frac{1
+(\theta_2-1)\frac{I_t}{K_t}}{1+\theta_2\left(\frac{I_t}{K_t}\right)}\end{bmatrix}
$$
 
- the `psi2` will also be a three dimensional array 
$$
\psi_2 = 
\begin{bmatrix}
\frac{C_t}{K_t} + \frac{I_t}{K_t} - {\mathbf a}\\
1\\
\frac{MK_t}{MC_t}-{\mathbf a}
\end{bmatrix}
$$

- the `phi` will be a two dimensional array 
$$\phi=\begin{bmatrix} 
&\frac{K_{t+1}}{K_t} - \left[1 + \theta_2 \left({\frac {I_t} {K_t}}\right) \right]^{\theta_1} \exp( - \alpha_k + Z_t - {\frac 1 2} \mid \sigma_k \mid^2  + \sigma_k\cdot W_{t+1})\\
&Z_{t+1} - {\mathbb A} Z_t + \mathbb{B} W_{t+1}
\end{bmatrix}$$

In [5]:
def eq_ac(Var_t, Var_tp1, W_tp1, q, mode, *args):
    
    # Unpack model parameters
    γ, β, ρ, a, ϕ_1, ϕ_2, α_k, σ_k, A, B = args

    # Unpack model variables
    q_t, cmk_t, imk_t, mkmc_t, Z_t, gk_t = Var_t.ravel()
    q_tp1, cmk_tp1, imk_tp1, mkmc_tp1, Z_tp1, gk_tp1 = Var_tp1.ravel()

    # Intermedite varibles that facilitates computation
    sdf_ex = anp.log(β) - ρ*(cmk_tp1+gk_tp1-cmk_t)
    g_dep = -α_k + Z_t + σ_k.T@W_tp1 - 0.5 *(σ_k.T@σ_k).item()
    
    ## Forward-looking conditions
    psi1_1 = 0.
    psi1_2 = anp.exp(sdf_ex + mkmc_tp1 + gk_tp1 - anp.log(1.+ϕ_2 * anp.exp(imk_t)))
    psi1_3 = anp.exp(sdf_ex + mkmc_tp1 + gk_tp1 + anp.log(1.+(ϕ_2-1) * anp.exp(imk_t)) - anp.log(1.+ ϕ_2*anp.exp(imk_t)))

    psi2_1 = -a + anp.exp(cmk_t) + anp.exp(imk_t)
    psi2_2 = 1.
    psi2_3 = anp.exp(mkmc_t) - a

    # State evoluion processes
    phi_1 = Z_tp1 - A*Z_t - B@W_tp1
    phi_2 = gk_tp1 - ϕ_1 * anp.log(1.+ϕ_2*anp.exp(imk_t)) - g_dep
    
    if mode == 'psi1':
        return np.array([psi1_1, psi1_2, psi1_3])
    
    return anp.array([
        psi1_1 * anp.exp(q_tp1) - psi2_1,
        psi1_2 * anp.exp(q_tp1) - psi2_2,
        psi1_3 * anp.exp(q_tp1) - psi2_3,
        phi_1, phi_2])

## 1.2 Model Steady State Function 

This function takes model parameters as input and outputs the steady states for each variable following the variable order specified in `Var_t` and `Var_tp1`.

- The first one will always be 0, since the steady state of `q_t` is 0.

In [6]:
def ss_ac(*args):

    γ, β, ρ, a, ϕ_1, ϕ_2, α_k, σ_k, A, B = args

    def f(imk):
        g_k = ϕ_1 * np.log(1.+ ϕ_2 * np.exp(imk)) - α_k - 0.5 *(σ_k.T@σ_k).item()
        sdf_ex = np.log(β) - ρ*g_k
        g_dep = -α_k - 0.5 *(σ_k.T@σ_k).item()
        mkmc = anp.log(1./anp.exp(sdf_ex  + g_k - anp.log(1.+ϕ_2 * anp.exp(imk))) )
        return anp.exp(sdf_ex + mkmc + g_k + anp.log(1.+(ϕ_2-1) * anp.exp(imk)) - anp.log(1.+ ϕ_2*anp.exp(imk))) - anp.exp(mkmc) + a

    imk = optimize.bisect(f,-40,np.log(a))
    cmk = np.log(a - np.exp(imk))
    g_k = ϕ_1 * np.log(1. + ϕ_2 * np.exp(imk)) - α_k - 0.5 *(σ_k.T@σ_k).item()
    sdf_ex = np.log(β) - ρ*g_k
    g_dep = - α_k - 0.5 *(σ_k.T@σ_k).item()
    mkmc = anp.log(1./anp.exp(sdf_ex  + g_k - anp.log(1.+ϕ_2 * anp.exp(imk))) )

    return np.array([0., cmk, imk, mkmc, 0., g_k])

## 1.3 Model Variable Dimension Array

Specify a simple numpy array for `(n_J, n_X, n_W)`. The dimensions for jump variables, state variables, and shock variables in the model.

In [7]:
var_shape = (3, 2, 2)

## 1.4 Model Parameter Calibration Tuple

Specify all the model parameters using a Python tuple. The first three are fixed recursive utility parameters, γ, β, ρ. 

In [8]:
γ = 10.
ρ = 4./3
β = np.exp(-0.0025)
a = 0.0288
ϕ_2 = 88.
ϕ_1 = 1 / ϕ_2 
α_k = 0.0088
σ_k = np.array([[0.477],[0]]) * 0.01
A = np.exp(-0.014)
B = np.array([[0.011,0.025]]) * 0.01

args = (γ, β, ρ, a, ϕ_1, ϕ_2, α_k, σ_k, A, B)

## 1.5 Log Consumption Growth Function

The expansion suite deals with the recursive utility in equation (2). We need to specify how to compute the log-growth of consumption using the defined variables `Var_t` and `Var_tp1` in the equilibrium conditions. 

- For some models with habits, $C_t$ is replaced with $U_t$, then we need to specify the log growth process for $U_t$ correspondingly.
- For some models with preference shocks, $C_t = \lambda_t \tilde{C}_t$, we need to specify the log growth process for $\lambda_t \tilde{C}_t$. 

$$
V_t=\left[(1-\beta)\left(C_t\right)^{1-\rho}+\beta\left(R_t\right)^{1-\rho}\right]^{\frac{1}{1-\rho}}\tag{2}
$$

In [9]:
def gc_ac(Var_t, Var_tp1, W_tp1, q, *args):
    
    # Unpack model parameters
    γ, β, ρ, a, ϕ_1, ϕ_2, α_k, σ_k, A, B = args

    # Unpack model variables (q_t and q_tp1 should be the first variables)
    q_t, cmk_t, imk_t, mkmc_t, Z_t, gk_t = Var_t.ravel()
    q_tp1, cmk_tp1, imk_tp1, mkmc_tp1, Z_tp1, gk_tp1 = Var_tp1.ravel()

    # Compute log consumption growth
    gc_tp1 = cmk_tp1 + gk_tp1 - cmk_t
    
    return gc_tp1

## 1.6 Solve the BY model using the Expansion Suite 

The expansion suite takes the above five necessary inputs to solve a model. The solution is stored in a class named `ModelSolution`(Please refer to uncertainexpansion.ipynb for details.) under the Linear Quadratic Framework. The computation of the framework is built on a `LinQuadVar` class.

In [10]:
# Solve BY model
eq = eq_ac
ss = ss_ac
gc = gc_ac
var_shape = (3, 2, 2)
args = (γ, β, ρ, a, ϕ_1, ϕ_2, α_k, σ_k, A, B)
approach = '1' # See Exploring Recursive Utility Appendix for difference about approach '1' and '2'
ModelSol = uncertain_expansion(eq, ss, var_shape, args, gc, approach)

Iteration 1: error = 1.13750867
Iteration 2: error = 0


## 1.7 Model Solutions

Below cells display analytical forms of the approximation results for jump variables and state variables. These variables are all stored as `LinQuadVar`. 

In [11]:
## Log consumption over capital approximation results, shown in the LinQuadVar coefficients form
ModelSol['JX1_t'][0].coeffs

{'x': array([[11.87510894, -0.        ]]), 'c': array([[-0.09142881]])}

In [12]:
## Log consumption over capital approximation results, shown in the Latex analytical form
disp(ModelSol['JX_t'][0],'\\log\\frac{C_t}{K_t}') 

<IPython.core.display.Math object>

In [13]:
## Log consumption over capital zeroth order approximation results
disp(ModelSol['J0_t'][0],'\\log\\frac{C_t^0}{K_t^0}')

<IPython.core.display.Math object>

In [14]:
## Log consumption over capital first order approximation results
disp(ModelSol['J1_t'][0],'\\log\\frac{C_t^1}{K_t^1}')

<IPython.core.display.Math object>

In [15]:
## Log consumption over capital second order approximation results
disp(ModelSol['J2_t'][0],'\\log\\frac{C_t^2}{K_t^2}')

<IPython.core.display.Math object>

In [16]:
## Log capital growth process first order approximation results
disp(ModelSol['X1_tp1'][0],'\\log\\frac{K_{t+1}^1}{K_t^1}') 

<IPython.core.display.Math object>

In [17]:
## Log capital growth process second order approximation results
disp(ModelSol['X2_tp1'][0],'\\log\\frac{K_{t+1}^2}{K_t^2}') 

<IPython.core.display.Math object>

## 1.8 Approximate Variables

If some variables can be computed using solved model variables, we can write a function to specify how to approximate, and then use the method `ModelSolution.approximate` to compute the approximation of these variables using model solutions. The below example computes the approximation of log investment growth process.

In [18]:
def gi_tp1_approx(Var_t, Var_tp1, W_tp1, q, *args):

    # Parameters for the model
    γ, β, ρ, a, ϕ_1, ϕ_2, α_k, σ_k, A, B = args

    # Variables: (q_t, q_tp1 is excluded when using the method in `ModelSolution`)
    cmk_t, imk_t, mkmc_t, Z_t, gk_t = Var_t.ravel()
    cmk_tp1, imk_tp1, mkmc_tp1, Z_tp1, gk_tp1 = Var_tp1.ravel()
    
    gi_tp1 = imk_tp1 + gk_tp1 -imk_t
    
    return gi_tp1

n_J, n_X, n_W = var_shape
gi_tp1_list = ModelSol.approximate(gi_tp1_approx, args)
gi_tp1 = {'gi_tp1':gi_tp1_list[0],\
        'gi0_tp1':gi_tp1_list[1],\
        'gi1_tp1':gi_tp1_list[2],\
        'gi2_tp1':gi_tp1_list[3]}
gi_tp1['gi1_tp1'].coeffs

{'x': array([[ 1.03727514, -0.        ]]),
 'w': array([[ 0.00415864, -0.00138945]]),
 'c': array([[0.00030791]])}

## 1.9 Model Simulations
Given a series of shock processes, we can simulate variables using the `ModelSolution.simulate` method. The below example simulates all jump and state variables with 400 periods i.i.d standard multivariate normal shocks.

In [16]:
Ws = np.random.multivariate_normal(np.zeros(n_W),np.eye(n_W),size = 400)
JX_sim = ModelSol.simulate(Ws)

## 1.10 Shock Elasticities and IRF

`ModelSolution` has a method `elasticities` to compute the shock elasticities of all jump and state variables by defining a `log_SDF_ex` function, which states the log stochastic discount factor exclusive of the change of measure ${N}^*_{t+1}$ and variable $Q_{t+1}^*$. Shock elasticities are great tools for nonlinear impulsive response analyses, and in linear models, the shock elasticities correspond to the traditional IRF. 

`ModelSolution` also has a method `IRF` to compute the traditional IRF of all jump and state variables. Below we calculate the IRF of all variables with respect to the first shock for 400 periods.

In [23]:
def log_SDF_ex(Var_t, Var_tp1, W_tp1, q, *args):

    # Log stochastic discount factor exclusive of the change of measure N and Q.

    # Parameters for the model 
    γ, β, ρ, a, ϕ_1, ϕ_2, α_k, σ_k, A, B = args

    # Variables: (q_t, q_tp1 is excluded when using the method in `ModelSolution`) 
    cmk_t, imk_t, mkmc_t, Z_t, gk_t = Var_t.ravel()
    cmk_tp1, imk_tp1, mkmc_tp1, Z_tp1, gk_tp1 = Var_tp1.ravel()
    
    sdf_ex = anp.log(β) - ρ*(cmk_tp1+gk_tp1-cmk_t)
    
    return sdf_ex

expo_elas, price_elas = ModelSol.elasticities(log_SDF_ex, T=400, shock=0)
states, controls = ModelSol.IRF(T = 400, shock = 0)

# 2 Examples to use `LinQuadVar` in Computation

## 2.1 Define a `LinQuadVar`

Below we extract the steady states for the state evolution processes from the previous model solution, and construct it as a 2-dimensional LinQuadVar.

In [18]:
n_J, n_X, n_W = ModelSol['var_shape']
X0_tp1 = LinQuadVar({'c':np.array([[ModelSol['ss'][1]],[ModelSol['ss'][2]]])}, shape = (2, n_X, n_W))
X0_tp1.coeffs

{'c': array([[-3.93129532],
        [ 1.00634613]])}

## 2.2 `LinQuadVar` Operations
Below show two ways to add two LinQuads, we use the zeroth order, first order, second order approximation results to construct the approximation for the growth process or both state evolution processes.

In [19]:
gk_tp1 = X0_tp1[0] + ModelSol['X1_tp1'][0]  + 0.5 * ModelSol['X2_tp1'][0] 
disp(gk_tp1,'\\log\\frac{K_{t+1}}{K_t}') 

<IPython.core.display.Math object>

In [20]:
lq_sum([X0_tp1, ModelSol['X1_tp1'], 0.5 * ModelSol['X2_tp1']]).coeffs

{'xx': array([[ 0.        ,  0.        ,  0.        ,  0.        ],
        [-0.21924476, -0.        ,  0.        ,  0.        ]]),
 'x': array([[0.98609754, 0.        ],
        [0.9628084 , 0.        ]]),
 'x2': array([[0.49304877, 0.        ],
        [0.48000401, 0.        ]]),
 'w': array([[0.00011, 0.00025],
        [0.00477, 0.     ]]),
 'c': array([[-3.93129532],
        [ 1.00667219]])}

## 2.3 `LinQuadVar` Split and Concat
`split` breaks multiple dimensional LinQuad into one-dimensional LinQuads, while `concat` does the inverse operation

In [21]:
gk_tp1, Z_tp1 = ModelSol['X1_tp1'].split()
concat([gk_tp1, Z_tp1])

## 2.4 Evaluate a `LinQuadVar`
Given specific values for $X_{t}$, $W_{t+1}$, we can evaluate a `LinQuadVar`. Below evaluates all variables at first and second order expansion states equal to 0, together with a multivariate random normal shock.

In [28]:
x1 = np.zeros([n_X ,1])
x2 = np.zeros([n_X ,1])
w = np.random.multivariate_normal(np.zeros(n_W),np.eye(n_W),size = 1).T
ModelSol['JX_tp1'](*(x1,x2,w))

array([[-4.78778195],
       [-3.88836269],
       [ 1.03332566],
       [ 0.00040123],
       [ 0.00032696]])

## 2.5 Adjust Conditional Information Time 
Since some `LinQuadVar` (e.g. solved jump variables) are expressed as a function of time $t$ state variables without shocks, when we would like to compute its next period expression, $t+1$, with shocks, but still as a function of time $t$ state variables, we can use function `next_period`.

In [23]:
cmk1_tp1 = next_period(ModelSol['J1_t'][0], ModelSol['X1_tp1'])
disp(cmk1_tp1, '\\log\\frac{C_{t+1}^1}{K_{t+1}^1}') 

<IPython.core.display.Math object>

In [24]:
cmk2_tp1 = next_period(ModelSol['J2_t'][0], ModelSol['X1_tp1'], ModelSol['X2_tp1'])
disp(cmk2_tp1, '\\log\\frac{C_{t+1}^2}{K_{t+1}^2}') 

<IPython.core.display.Math object>

## 2.6 Compute the Expectation of time $t+1$ `LinQuadVar`

Suppose the distribution of shock has a constant mean and variance (not state dependent), we can use `E` to compute the expectation of time $t+1$ `LinQuadVar`.


In [25]:
E_w = ModelSol['util_sol']['μ_0']
cov_w = np.eye(n_W)
E_ww = cal_E_ww(E_w, cov_w)
E_cmk2_tp1 = E(cmk2_tp1, E_w, E_ww)
disp(E_cmk2_tp1, '\mathbb{E}[\\log\\frac{C_{t+1}^2}{K_{t+1}^2}|\mathfrak{F_t}]')

<IPython.core.display.Math object>

Suppose the distribution of shock has a state-dependent mean and variance (implied by $\tilde{N}_{t+1}$ shown in the notes), we can use `E_N_tp1` and `N_tilde_measure` to compute the expectation of time $t+1$ `LinQuadVar`.

In [26]:
N_cm = N_tilde_measure(ModelSol['util_sol']['log_N_tilde'],(1,n_X,n_W))
E_cmk2_tp1_tilde = E_N_tp1(cmk2_tp1, N_cm)
disp(E_cmk2_tp1_tilde, '\mathbb{\\tilde{E}}[\\log\\frac{C_{t+1}^2}{K_{t+1}^2}|\mathfrak{F_t}]')

<IPython.core.display.Math object>